In [1]:
# imports

import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import torch.optim as optim

In [2]:
#reference pytorch

np.arange(3.0)

x = torch.empty(5,3) # will have some junk
print(x[0,0])

torch.zeros_like(x)
torch.zeros(5,3)

y = torch.rand(5,3)
print(y)

x.size()

torch.tensor([2,1,4])

torch.add(x,y)

z = torch.tensor([1.234])
z.item() #will work for one item to extract out

tensor(-5.6816e-14)
tensor([[0.3037, 0.8901, 0.2762],
        [0.5385, 0.5139, 0.0783],
        [0.2564, 0.1407, 0.2993],
        [0.8948, 0.1888, 0.4091],
        [0.6919, 0.8999, 0.9015]])


1.2339999675750732

In [ ]:
# task1

In [ ]:
# way
-> Took whole file as single dim for ds1

In [3]:
# input data

path = '11_ds_1/'
class_names = os.listdir(path)
print(class_names)
data = []
target = []

for cn in range(len(class_names)):
    for fname in os.listdir(path+class_names[cn]+'/'):
        data.append(np.loadtxt(path+class_names[cn]+'/'+fname).flatten())
        target.append(int(cn))

['forest', 'mountain', 'coast', 'insidecity', 'street']


In [4]:
# parameters

dim = 500
h1_dim = 40
h2_dim = 28
out_dim = len(class_names)
learning_rate = 0.001
momentum = 0.9
num_epochs = 500
batch_size = 32  # general rule is to try 32, 64, 128 ... 

In [5]:
# test train split

train_data, test_data, train_target, test_target = train_test_split(data,target,test_size=0.2)

In [6]:
# PCA

pca = decomposition.PCA(n_components = dim)
pca.fit(train_data)
print(np.shape(train_data))
train_data = pca.transform(train_data)
test_data = pca.transform(test_data)
print(np.shape(train_data))

(1329, 828)
(1329, 500)


In [7]:
# NN model

class Nural_net(nn.Module):
    def __init__(self):
        super(Nural_net, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fc3 = nn.Linear(h2_dim,out_dim)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# loss fun and update method

tensor_train_data = torch.Tensor(train_data)
tensor_train_target = torch.LongTensor(train_target) # long otherwise takes as float
 
tensor_test_data = torch.Tensor(test_data)
tensor_test_target = torch.LongTensor(test_target)

# num_workers - how many parallel data loadings
train_dataset = torch.utils.data.TensorDataset(tensor_train_data,tensor_train_target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=4)

test_dataset = torch.utils.data.TensorDataset(tensor_test_data,tensor_test_target)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False,num_workers=4)

net = Nural_net()
criterion = nn.CrossEntropyLoss()
# ASDG for generalised delta ig
# schochastic gradient descent with nestrov momentum
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate,lr_decay=0)

In [9]:
# training


running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(train_loader):
        
        inputs, labels = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

50 1.5743935062771752
100 1.3395341465870538
150 1.115949059781574
200 0.9349301714272726
250 0.8097666255633036
300 0.702774885963826
350 0.6030406593141102
400 0.5177058787289119
450 0.4440224064744654
500 0.37910610306830633


In [15]:
# save 
# model1 is saved after fine tuning

saved_net_path = './net_model2.pth'
torch.save(net.state_dict(),saved_net_path)

In [16]:
# retrive saved net

test_net_path = './net_model2.pth'
test_net = Nural_net()
test_net.load_state_dict(torch.load(test_net_path))

<All keys matched successfully>

In [17]:
# testing

# train data
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        outputs = test_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('train accuracy',end=':')
print(correct/total)

#test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = test_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('test accuracy',end=':')
print(correct/total)

train accuracy:0.9006772009029346
test accuracy:0.7447447447447447


In [ ]:
dim = 828
h1_dim = 1000
h2_dim = 500
batch_size = 32
learning_rate = 0.001
num_epochs = 500

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(data,target,test_size=0.2)

In [ ]:
# AANN

class AANN(nn.Module):
    def __init__(self):
        super(AANN, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fc3 = nn.Linear(h2_dim,h1_dim)
        self.fc4 = nn.Linear(h1_dim,dim)
        
    def forward(self, x):
        m = nn.Tanh()
        x = self.fc1(x)
        y = m(self.fc2(x))
        x = m(self.fc3(y))
        x = self.fc4(x)
        return x, y

In [ ]:
# training for AANN

tensor_data = torch.Tensor(train_data)

dataset = torch.utils.data.TensorDataset(tensor_data,tensor_data)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=4)

aann = AANN()
criterion = nn.MSELoss(size_average=None, reduce=None, reduction='mean')
optimizer = optim.Adagrad(aann.parameters(), lr=learning_rate,lr_decay=0)

In [ ]:
running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(loader):
        
        inputs, same_inputs = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs,_ = aann(inputs)
        loss = criterion(outputs,same_inputs)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

In [ ]:
error = 0
num = 0
with torch.no_grad():
    for data in loader:
        inputs, labels = data
        outputs,_ = aann(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('error',end=':')
print(error/(num*dim))

In [ ]:
saved_net = './aann_model2.pth'
torch.save(aann.state_dict(),saved_net)

In [ ]:
aann_train = torch.Tensor(train_data)
aann_test = torch.Tensor(test_data)

In [ ]:
with torch.no_grad():
    print(aann_train.size())
    _, aann_train = aann(aann_train)
    _, aann_test = aann(aann_test)
    print(aann_train.size())

In [ ]:
# parameters

dim = 500
h1_dim = 40
h2_dim = 28
out_dim = len(class_names)
learning_rate = 0.001
momentum = 0.9
num_epochs = 1000
batch_size = 32  # general rule is to try 32, 64, 128 ... 

In [ ]:
# loss fun and update method

tensor_train_target = torch.LongTensor(train_target) # long otherwise takes as float

tensor_test_target = torch.LongTensor(test_target)

# num_workers - how many parallel data loadings
train_dataset = torch.utils.data.TensorDataset(aann_train,tensor_train_target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=4)

test_dataset = torch.utils.data.TensorDataset(aann_test,tensor_test_target)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False,num_workers=4)

net2 = Nural_net()
criterion = nn.CrossEntropyLoss()
# ASDG for generalised delta ig
# schochastic gradient descent with nestrov momentum
optimizer = optim.SGD(net2.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate,lr_decay=0)

In [ ]:
# training


running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(train_loader):
        
        inputs, labels = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs = net2(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

In [ ]:
# testing

# train data
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        outputs = net2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('train accuracy',end=':')
print(correct/total)

#test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('test accuracy',end=':')
print(correct/total)

In [ ]:
saved_net = './net2_model2.pth'
torch.save(net2.state_dict(),saved_net)